# Efficient Yelp API Calls (Core)
Student Name: Nian

In [1]:
# Imports
import numpy as np
import pandas as pd
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Key Variables
# Change these to control the program
secret_file = '/Users/nianv/.secret/yelp_api.json' # Yelp API Kyes
json_file = 'Data/results_workfile.json' # Workfile for results of search
output_csv = 'Data/results.csv' # CSV File to save results in
overwrite_workfile = True # Overwrite Workfile if exists?
location = 'NY,NY' # Search location
search_term = 'hot dogs' # Search term
api_limit = 1000 # Yelp API Limit

In [3]:
def json_load(file=json_file):
    with open(file, 'r') as f:
        return json.load(f)

def json_dump(file=json_file, dump_content=[]):
    with open(file, 'w') as f:
        json.dump(dump_content, f)

In [4]:
with open(secret_file) as f:
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
file_exists = os.path.isfile(json_file)

if not file_exists:
    folder = os.path.dirname(json_file)
    if len(folder) > 0:
        os.makedirs(folder, exist_ok=True)

    print(f'[i] {json_file} not found. Saving empty list to file.')

    json_dump(json_file, [])
else:
    if overwrite_workfile == True:
        json_dump(json_file, [])
        print(f'[i] Existing file {json_file} found. File Overwritten.')
    else:
        print(f'[i] Existing file {json_file} found. File not Overwritten.')

[i] Existing file Data/results_workfile.json found. File Overwritten.


In [6]:
previous_results = json_load(json_file)

n_results = len(previous_results)
print(f'[i] {n_results} previous results found.')

[i] 0 previous results found.


In [7]:
results = yelp_api.search_query(location=location, term=search_term, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
previous_results.extend(results['businesses'])
json_dump(json_file, previous_results)

In [9]:
results_per_page = len(results['businesses'])
print(f"[i] Total Results: {results['total']}\n[i] Results per Page: {results_per_page}")

[i] Total Results: 4100
[i] Results per Page: 20


In [10]:
n_pages = math.ceil((results['total']-n_results/results_per_page))
print(f'[i] Total Results: {n_pages}')

[i] Total Results: 4100


In [11]:
# Get amount of pages that will be retrieved
if n_pages * results_per_page <= api_limit:
    pages_to_retrieve = n_pages
else:
    pages_to_retrieve = api_limit // results_per_page
    print('[i] Reducing pages to fetch due to api limit')

print(f'[i] Total Pages of Records: {n_pages}\n[i] Pages to Retrieve: {pages_to_retrieve}')

# Retrieve all pages, buffer each api call with .2 sec, save to workfile
for i in tqdm_notebook(range(0, pages_to_retrieve)):
    if i == 0:
        continue # Account for previous run
        
    previous_results = json_load(json_file)

    n_results = len(previous_results)

    if (n_results + results_per_page) > api_limit:
        print(f'Exceeded {api_limit} api calls. Stopping loop')
        break
    else:
        time.sleep(.2)

    results = yelp_api.search_query(location=location, term=search_term, offset=n_results)
    
    previous_results.extend(results['businesses'])
    
    json_dump(json_file, previous_results)

[i] Reducing pages to fetch due to api limit
[i] Total Pages of Records: 4100
[i] Pages to Retrieve: 50


  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
results_df = pd.DataFrame(previous_results)
results_df.to_csv(output_csv)
results_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,2n01u4yKfIrWuOzRVvIZmw,crif-dogs-new-york,Crif Dogs,https://s3-media1.fl.yelpcdn.com/bphoto/9wzdpD...,False,https://www.yelp.com/biz/crif-dogs-new-york?ad...,2140,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",4.0,"{'latitude': 40.72707, 'longitude': -73.98379}","[pickup, delivery]",$,"{'address1': '113 Saint Marks Pl', 'address2':...",+16469228524,(646) 922-8524,2567.471632
1,exkTMrGqV-4j_z0QRVVY7Q,billys-hot-dog-cart-new-york,Billy's Hot Dog Cart,https://s3-media3.fl.yelpcdn.com/bphoto/zP-jch...,False,https://www.yelp.com/biz/billys-hot-dog-cart-n...,78,"[{'alias': 'foodstands', 'title': 'Food Stands...",5.0,"{'latitude': 40.789695, 'longitude': -73.966125}",[],$,"{'address1': '327 Central Park West', 'address...",,,9660.221018
2,477o_-7VmcnTokazWtEa0A,grays-papaya-new-york,Gray's Papaya,https://s3-media3.fl.yelpcdn.com/bphoto/zF4W7-...,False,https://www.yelp.com/biz/grays-papaya-new-york...,1529,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",3.5,"{'latitude': 40.7783783121866, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '2090 Broadway', 'address2': None...",+12127990243,(212) 799-0243,8180.482318
3,EL5SxC51am9wSTKN0xtCyQ,glizzys-nyc-brooklyn,Glizzy's NYC,https://s3-media1.fl.yelpcdn.com/bphoto/fX2w9v...,False,https://www.yelp.com/biz/glizzys-nyc-brooklyn?...,12,"[{'alias': 'comfortfood', 'title': 'Comfort Fo...",4.0,"{'latitude': 40.713908, 'longitude': -73.955302}","[pickup, delivery]",NaN,"{'address1': '390 Metropolitan Ave', 'address2...",,,3411.679721
4,ydYyMb4mmnJ0--4ZE_MsjA,dog-day-afternoon-brooklyn,Dog Day Afternoon,https://s3-media3.fl.yelpcdn.com/bphoto/HS77fR...,False,https://www.yelp.com/biz/dog-day-afternoon-bro...,69,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}]",4.5,"{'latitude': 40.6586, 'longitude': -73.98213}","[pickup, delivery]",$,"{'address1': '266 Prospect Park W', 'address2'...",,,5304.288793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,cRwpZQaoaPsdG2tAYY7-8Q,mango-and-bananas-cafe-west-new-york,Mango & Bananas Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/LIhnGn...,False,https://www.yelp.com/biz/mango-and-bananas-caf...,10,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 40.7824911767414, 'longitude': -7...","[delivery, pickup]",NaN,"{'address1': '4909 Bergenline Ave', 'address2'...",+12014304865,(201) 430-4865,8866.985000
996,h0BOGvJ-C4FdnsrBXbIj3A,au-jus-new-york-8,Au Jus,https://s3-media1.fl.yelpcdn.com/bphoto/wiOP_v...,False,https://www.yelp.com/biz/au-jus-new-york-8?adj...,56,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 40.788558, 'longitude': -73.948589}","[pickup, delivery]",$,"{'address1': '1569 Lexington Ave', 'address2':...",+12123607200,(212) 360-7200,10012.546097
997,bxJh48jROb-KKy9rPUrHjQ,eating-tree-bronx,Eating Tree,https://s3-media2.fl.yelpcdn.com/bphoto/C9BlNi...,False,https://www.yelp.com/biz/eating-tree-bronx?adj...,148,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",3.5,"{'latitude': 40.8281599, 'longitude': -73.92442}","[delivery, pickup]",$,"{'address1': '892 Gerard Ave', 'address2': '',...",+17182935025,(718) 293-5025,14857.125707
998,5VMaxD9JBGZTqacKU6yCSw,peter-luger-steak-house-great-neck-2,Peter Luger Steak House,https://s3-media4.fl.yelpcdn.com/bphoto/axmRpo...,False,https://www.yelp.com/biz/peter-luger-steak-hou...,1433,"[{'alias': 'steak', 'title': 'Steakhouses'}]",4.0,"{'latitude': 40.777507, 'longitude': -73.727721}",[delivery],$$$$,"{'address1': '255 Northern Blvd', 'address2': ...",+15164878800,(516) 487-8800,23848.397613
